## Intro
This notebook describes how to prepare data for RNN/LSTM for timeseries prediction problem especially for Keras/Tensorflow

In [1]:
import numpy as np
import itertools

np.set_printoptions(suppress=True) # to suppress scientific notation while printing arrays

### Create Data
Create a data which is supposed to represent a timeseries prediction problem. The data has 6 columns and 1000 columns. The first five columns are supposed to be input and the last column is supposed to be output.

In [2]:
rows = 2000
cols = 6
data = np.arange(int(rows*cols)).reshape(-1,rows).transpose()
print(data[0:20])  
print('\n {} \n'.format(data.shape))
print(data[-20:])

[[    0  2000  4000  6000  8000 10000]
 [    1  2001  4001  6001  8001 10001]
 [    2  2002  4002  6002  8002 10002]
 [    3  2003  4003  6003  8003 10003]
 [    4  2004  4004  6004  8004 10004]
 [    5  2005  4005  6005  8005 10005]
 [    6  2006  4006  6006  8006 10006]
 [    7  2007  4007  6007  8007 10007]
 [    8  2008  4008  6008  8008 10008]
 [    9  2009  4009  6009  8009 10009]
 [   10  2010  4010  6010  8010 10010]
 [   11  2011  4011  6011  8011 10011]
 [   12  2012  4012  6012  8012 10012]
 [   13  2013  4013  6013  8013 10013]
 [   14  2014  4014  6014  8014 10014]
 [   15  2015  4015  6015  8015 10015]
 [   16  2016  4016  6016  8016 10016]
 [   17  2017  4017  6017  8017 10017]
 [   18  2018  4018  6018  8018 10018]
 [   19  2019  4019  6019  8019 10019]]

 (2000, 6) 

[[ 1980  3980  5980  7980  9980 11980]
 [ 1981  3981  5981  7981  9981 11981]
 [ 1982  3982  5982  7982  9982 11982]
 [ 1983  3983  5983  7983  9983 11983]
 [ 1984  3984  5984  7984  9984 11984]
 [ 1985  3

In [3]:
def first_nan_from_end(ar):
    """ 
    This function finds index for first nan from the group which is present at the end of array.
    [np.nan, np.nan, 0,2,3,0,3, np.nan, np.nan, np.nan, np.nan] >> 7
    [np.nan, np.nan, 1,2,3,0, np.nan, np.nan, np.nan] >> 6
    [0,2,3,0,3] >> 5
    [np.nan, np.nan, 0,2,3,0,3] >> 7    
    """
    last_non_zero=0
    
    for idx, val in enumerate(ar[::-1]):
        if ~np.isnan(val): # val >= 0:
            last_non_zero = idx
            break
    return ar.shape[0] - last_non_zero    
    

def batch_generator(data, lookback, in_features, out_features, batch_size, step, min_ind, max_ind, future_y_val,
                   norm=None, trim_last_batch=True):
    """
    :param data: `ndarray`, input data.
    :param lookback: `int`, sequence length, number of values LSTM will see at time `t` to make prediction at `t+1`.
    :in_features: `int`, number of columns in `data` starting from 0 to be considered as input
    :out_features: `int`, number of columns in `data` started from last to be considred as output/prediction.
    :parm norm: a dictionary which contains scaler object with which to normalize x and y data. We use separate scalers for x
                 and y data. Keys must be `x_scaler` and `y_scaler`.
    :parm trim_last_batch: bool, if True, last batch will be ignored if that contains samples less than `batch_size`.
    """
    
    # selecting the data of interest for x and y    
    X = data[min_ind:max_ind, 0:in_features]
    Y = data[min_ind:max_ind, -out_features:].reshape(-1,out_features)
    
    if norm:
        x_scaler = norm['x_scaler']
        y_scaler = norm['y_scaler']
        X = x_scaler.fit_transform(X)
        Y = y_scaler.fit_transform(Y)        
    
    # container for keeping x and y windows. A `windows` is here defined as one complete set of data at one timestep.
    x_wins = np.full((X.shape[0], lookback, in_features), np.nan, dtype=np.float32)
    y_wins = np.full((Y.shape[0], out_features), np.nan)
    
    # creating windows from X data
    st = lookback*step - step                 # starting point of sampling from data
    for j in range(st, X.shape[0]-lookback):
        en = j - lookback*step
        indices = np.arange(j, en, -step)
        ind = np.flip(indices)
        x_wins[j,:,:] = X[ind,:]

    # creating windows from Y data
    for i in range(0, Y.shape[0]-lookback):
        y_wins[i,:] = Y[i+lookback,:]



    """removing trailing nans"""
    first_nan_at_end = first_nan_from_end(y_wins[:,0])  # first nan in last part of data, start skipping from here
    y_wins = y_wins[0:first_nan_at_end,:]
    x_wins = x_wins[0:first_nan_at_end,:]

    """removing nans from start"""
    y_val = st-lookback + future_y_val
    if st>0:
        x_wins = x_wins[st:,:]
        y_wins = y_wins[y_val:,:]


    print("""shape of x data: {} \nshape of y data: {}""".format(x_wins.shape, y_wins.shape))


    print(""".\n{} values are skipped from start and {} values are skipped from end in output array"""
          .format(st, X.shape[0]-first_nan_at_end))

    pot_samples = x_wins.shape[0]

    print('\npotential samples are {}'.format(pot_samples))

    residue = pot_samples % batch_size
    print('\nresidue is {} '.format(residue))

    samples = pot_samples - residue
    print('\nActual samples are {}'.format(samples))

    interval = np.arange(0, samples + batch_size, batch_size)
    print('\nPotential intervals: {}'.format(interval ))

    if residue > 0:
        interval = np.append(interval, pot_samples)
    print('\nActual interval: {} '.format(interval))

    if trim_last_batch:
        no_of_batches = len(interval)-2
    else:
        no_of_batches = len(interval) - 1 
        
    print('\nNumber of batches are {} '.format(no_of_batches))


    x_batches = np.full((no_of_batches, batch_size, lookback, in_features), np.nan)
    y_batches = np.full((no_of_batches, batch_size, out_features), np.nan)


    for b in range(no_of_batches):
        st = interval[b]
        en = interval[b + 1]
        an_x_batch = x_wins[st:en, :, :]
        x_batches[b] = an_x_batch
       # y_batches[b] = y_wins[st:en]
        y_batches[b] = y_wins[st+1:en+1]


    print('\nshape of batches for:')
    print('x_data ', ' y_data')
    for i,j in zip(x_batches, y_batches):
        ishp, jshp = None, None
        if isinstance(i, np.ndarray):
            ishp = i.shape
        if isinstance(j, np.ndarray):
            jshp = j.shape
        print(ishp, jshp)
    
    return x_batches, y_batches




_lookback=7  # sequence length
input_features = 5
output_features = 1
_batch_size = 16
input_stepsize = 2
st_ind = 0
end_ind = 600
t_plus_ith_val = 1 # which value to predict in future, e.g if input is 11,12,13,14 and default value of this variable means we
                  # want to predict 15, setting value equal to 3 means we want to predict 17.

train_x_batches, train_y_batches = batch_generator(data, _lookback, input_features, output_features, _batch_size,
                                    input_stepsize, st_ind, end_ind, t_plus_ith_val,
                                    trim_last_batch = True)            

test_x_batches, test_y_batches = batch_generator(data, _lookback, input_features, output_features, _batch_size,
                                    input_stepsize,
                                    min_ind = 600,
                                    max_ind = 800,
                                    future_y_val = t_plus_ith_val,
                                    trim_last_batch = True)


shape of x data: (581, 7, 5) 
shape of y data: (587, 1)
.
12 values are skipped from start and 7 values are skipped from end in output array

potential samples are 581

residue is 5 

Actual samples are 576

Potential intervals: [  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272
 288 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560
 576]

Actual interval: [  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272
 288 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560
 576 581] 

Number of batches are 36 

shape of batches for:
x_data   y_data
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1)
(16, 7, 5) (16, 1

### first train batch

In [4]:
for inp,out in zip(train_x_batches[0], train_y_batches[0]):
    print(inp,out, '\n')

[[   0. 2000. 4000. 6000. 8000.]
 [   2. 2002. 4002. 6002. 8002.]
 [   4. 2004. 4004. 6004. 8004.]
 [   6. 2006. 4006. 6006. 8006.]
 [   8. 2008. 4008. 6008. 8008.]
 [  10. 2010. 4010. 6010. 8010.]
 [  12. 2012. 4012. 6012. 8012.]] [10014.] 

[[   1. 2001. 4001. 6001. 8001.]
 [   3. 2003. 4003. 6003. 8003.]
 [   5. 2005. 4005. 6005. 8005.]
 [   7. 2007. 4007. 6007. 8007.]
 [   9. 2009. 4009. 6009. 8009.]
 [  11. 2011. 4011. 6011. 8011.]
 [  13. 2013. 4013. 6013. 8013.]] [10015.] 

[[   2. 2002. 4002. 6002. 8002.]
 [   4. 2004. 4004. 6004. 8004.]
 [   6. 2006. 4006. 6006. 8006.]
 [   8. 2008. 4008. 6008. 8008.]
 [  10. 2010. 4010. 6010. 8010.]
 [  12. 2012. 4012. 6012. 8012.]
 [  14. 2014. 4014. 6014. 8014.]] [10016.] 

[[   3. 2003. 4003. 6003. 8003.]
 [   5. 2005. 4005. 6005. 8005.]
 [   7. 2007. 4007. 6007. 8007.]
 [   9. 2009. 4009. 6009. 8009.]
 [  11. 2011. 4011. 6011. 8011.]
 [  13. 2013. 4013. 6013. 8013.]
 [  15. 2015. 4015. 6015. 8015.]] [10017.] 

[[   4. 2004. 4004. 6004. 80

### second train batch

In [ ]:
for inp,out in zip(train_x_batches[1], train_y_batches[1]):
    print(inp,out, '\n')

[[  16. 2016. 4016. 6016. 8016.]
 [  18. 2018. 4018. 6018. 8018.]
 [  20. 2020. 4020. 6020. 8020.]
 [  22. 2022. 4022. 6022. 8022.]
 [  24. 2024. 4024. 6024. 8024.]
 [  26. 2026. 4026. 6026. 8026.]
 [  28. 2028. 4028. 6028. 8028.]] [10030.] 

[[  17. 2017. 4017. 6017. 8017.]
 [  19. 2019. 4019. 6019. 8019.]
 [  21. 2021. 4021. 6021. 8021.]
 [  23. 2023. 4023. 6023. 8023.]
 [  25. 2025. 4025. 6025. 8025.]
 [  27. 2027. 4027. 6027. 8027.]
 [  29. 2029. 4029. 6029. 8029.]] [10031.] 

[[  18. 2018. 4018. 6018. 8018.]
 [  20. 2020. 4020. 6020. 8020.]
 [  22. 2022. 4022. 6022. 8022.]
 [  24. 2024. 4024. 6024. 8024.]
 [  26. 2026. 4026. 6026. 8026.]
 [  28. 2028. 4028. 6028. 8028.]
 [  30. 2030. 4030. 6030. 8030.]] [10032.] 

[[  19. 2019. 4019. 6019. 8019.]
 [  21. 2021. 4021. 6021. 8021.]
 [  23. 2023. 4023. 6023. 8023.]
 [  25. 2025. 4025. 6025. 8025.]
 [  27. 2027. 4027. 6027. 8027.]
 [  29. 2029. 4029. 6029. 8029.]
 [  31. 2031. 4031. 6031. 8031.]] [10033.] 

[[  20. 2020. 4020. 6020. 80

### last train batch

In [ ]:
for inp,out in zip(train_x_batches[-1], train_y_batches[-1]):
    print(inp,out, '\n')

[[ 560. 2560. 4560. 6560. 8560.]
 [ 562. 2562. 4562. 6562. 8562.]
 [ 564. 2564. 4564. 6564. 8564.]
 [ 566. 2566. 4566. 6566. 8566.]
 [ 568. 2568. 4568. 6568. 8568.]
 [ 570. 2570. 4570. 6570. 8570.]
 [ 572. 2572. 4572. 6572. 8572.]] [10574.] 

[[ 561. 2561. 4561. 6561. 8561.]
 [ 563. 2563. 4563. 6563. 8563.]
 [ 565. 2565. 4565. 6565. 8565.]
 [ 567. 2567. 4567. 6567. 8567.]
 [ 569. 2569. 4569. 6569. 8569.]
 [ 571. 2571. 4571. 6571. 8571.]
 [ 573. 2573. 4573. 6573. 8573.]] [10575.] 

[[ 562. 2562. 4562. 6562. 8562.]
 [ 564. 2564. 4564. 6564. 8564.]
 [ 566. 2566. 4566. 6566. 8566.]
 [ 568. 2568. 4568. 6568. 8568.]
 [ 570. 2570. 4570. 6570. 8570.]
 [ 572. 2572. 4572. 6572. 8572.]
 [ 574. 2574. 4574. 6574. 8574.]] [10576.] 

[[ 563. 2563. 4563. 6563. 8563.]
 [ 565. 2565. 4565. 6565. 8565.]
 [ 567. 2567. 4567. 6567. 8567.]
 [ 569. 2569. 4569. 6569. 8569.]
 [ 571. 2571. 4571. 6571. 8571.]
 [ 573. 2573. 4573. 6573. 8573.]
 [ 575. 2575. 4575. 6575. 8575.]] [10577.] 

[[ 564. 2564. 4564. 6564. 85

### first test batch

In [ ]:
for inp,out in zip(test_x_batches[0], test_y_batches[0]):
    print(inp,out, '\n')

[[ 600. 2600. 4600. 6600. 8600.]
 [ 602. 2602. 4602. 6602. 8602.]
 [ 604. 2604. 4604. 6604. 8604.]
 [ 606. 2606. 4606. 6606. 8606.]
 [ 608. 2608. 4608. 6608. 8608.]
 [ 610. 2610. 4610. 6610. 8610.]
 [ 612. 2612. 4612. 6612. 8612.]] [10614.] 

[[ 601. 2601. 4601. 6601. 8601.]
 [ 603. 2603. 4603. 6603. 8603.]
 [ 605. 2605. 4605. 6605. 8605.]
 [ 607. 2607. 4607. 6607. 8607.]
 [ 609. 2609. 4609. 6609. 8609.]
 [ 611. 2611. 4611. 6611. 8611.]
 [ 613. 2613. 4613. 6613. 8613.]] [10615.] 

[[ 602. 2602. 4602. 6602. 8602.]
 [ 604. 2604. 4604. 6604. 8604.]
 [ 606. 2606. 4606. 6606. 8606.]
 [ 608. 2608. 4608. 6608. 8608.]
 [ 610. 2610. 4610. 6610. 8610.]
 [ 612. 2612. 4612. 6612. 8612.]
 [ 614. 2614. 4614. 6614. 8614.]] [10616.] 

[[ 603. 2603. 4603. 6603. 8603.]
 [ 605. 2605. 4605. 6605. 8605.]
 [ 607. 2607. 4607. 6607. 8607.]
 [ 609. 2609. 4609. 6609. 8609.]
 [ 611. 2611. 4611. 6611. 8611.]
 [ 613. 2613. 4613. 6613. 8613.]
 [ 615. 2615. 4615. 6615. 8615.]] [10617.] 

[[ 604. 2604. 4604. 6604. 86

### second test batch

In [ ]:
for inp,out in zip(test_x_batches[1], test_y_batches[1]):
    print(inp,out, '\n')

[[ 616. 2616. 4616. 6616. 8616.]
 [ 618. 2618. 4618. 6618. 8618.]
 [ 620. 2620. 4620. 6620. 8620.]
 [ 622. 2622. 4622. 6622. 8622.]
 [ 624. 2624. 4624. 6624. 8624.]
 [ 626. 2626. 4626. 6626. 8626.]
 [ 628. 2628. 4628. 6628. 8628.]] [10630.] 

[[ 617. 2617. 4617. 6617. 8617.]
 [ 619. 2619. 4619. 6619. 8619.]
 [ 621. 2621. 4621. 6621. 8621.]
 [ 623. 2623. 4623. 6623. 8623.]
 [ 625. 2625. 4625. 6625. 8625.]
 [ 627. 2627. 4627. 6627. 8627.]
 [ 629. 2629. 4629. 6629. 8629.]] [10631.] 

[[ 618. 2618. 4618. 6618. 8618.]
 [ 620. 2620. 4620. 6620. 8620.]
 [ 622. 2622. 4622. 6622. 8622.]
 [ 624. 2624. 4624. 6624. 8624.]
 [ 626. 2626. 4626. 6626. 8626.]
 [ 628. 2628. 4628. 6628. 8628.]
 [ 630. 2630. 4630. 6630. 8630.]] [10632.] 

[[ 619. 2619. 4619. 6619. 8619.]
 [ 621. 2621. 4621. 6621. 8621.]
 [ 623. 2623. 4623. 6623. 8623.]
 [ 625. 2625. 4625. 6625. 8625.]
 [ 627. 2627. 4627. 6627. 8627.]
 [ 629. 2629. 4629. 6629. 8629.]
 [ 631. 2631. 4631. 6631. 8631.]] [10633.] 

[[ 620. 2620. 4620. 6620. 86

### second last test batch

In [ ]:
for inp,out in zip(test_x_batches[-2], test_y_batches[-2]):
    print(inp,out, '\n')

[[ 744. 2744. 4744. 6744. 8744.]
 [ 746. 2746. 4746. 6746. 8746.]
 [ 748. 2748. 4748. 6748. 8748.]
 [ 750. 2750. 4750. 6750. 8750.]
 [ 752. 2752. 4752. 6752. 8752.]
 [ 754. 2754. 4754. 6754. 8754.]
 [ 756. 2756. 4756. 6756. 8756.]] [10758.] 

[[ 745. 2745. 4745. 6745. 8745.]
 [ 747. 2747. 4747. 6747. 8747.]
 [ 749. 2749. 4749. 6749. 8749.]
 [ 751. 2751. 4751. 6751. 8751.]
 [ 753. 2753. 4753. 6753. 8753.]
 [ 755. 2755. 4755. 6755. 8755.]
 [ 757. 2757. 4757. 6757. 8757.]] [10759.] 

[[ 746. 2746. 4746. 6746. 8746.]
 [ 748. 2748. 4748. 6748. 8748.]
 [ 750. 2750. 4750. 6750. 8750.]
 [ 752. 2752. 4752. 6752. 8752.]
 [ 754. 2754. 4754. 6754. 8754.]
 [ 756. 2756. 4756. 6756. 8756.]
 [ 758. 2758. 4758. 6758. 8758.]] [10760.] 

[[ 747. 2747. 4747. 6747. 8747.]
 [ 749. 2749. 4749. 6749. 8749.]
 [ 751. 2751. 4751. 6751. 8751.]
 [ 753. 2753. 4753. 6753. 8753.]
 [ 755. 2755. 4755. 6755. 8755.]
 [ 757. 2757. 4757. 6757. 8757.]
 [ 759. 2759. 4759. 6759. 8759.]] [10761.] 

[[ 748. 2748. 4748. 6748. 87

### Last test batch

In [ ]:
for inp,out in zip(test_x_batches[-1], test_y_batches[-1]):
    print(inp,out, '\n')

[[ 760. 2760. 4760. 6760. 8760.]
 [ 762. 2762. 4762. 6762. 8762.]
 [ 764. 2764. 4764. 6764. 8764.]
 [ 766. 2766. 4766. 6766. 8766.]
 [ 768. 2768. 4768. 6768. 8768.]
 [ 770. 2770. 4770. 6770. 8770.]
 [ 772. 2772. 4772. 6772. 8772.]] [10774.] 

[[ 761. 2761. 4761. 6761. 8761.]
 [ 763. 2763. 4763. 6763. 8763.]
 [ 765. 2765. 4765. 6765. 8765.]
 [ 767. 2767. 4767. 6767. 8767.]
 [ 769. 2769. 4769. 6769. 8769.]
 [ 771. 2771. 4771. 6771. 8771.]
 [ 773. 2773. 4773. 6773. 8773.]] [10775.] 

[[ 762. 2762. 4762. 6762. 8762.]
 [ 764. 2764. 4764. 6764. 8764.]
 [ 766. 2766. 4766. 6766. 8766.]
 [ 768. 2768. 4768. 6768. 8768.]
 [ 770. 2770. 4770. 6770. 8770.]
 [ 772. 2772. 4772. 6772. 8772.]
 [ 774. 2774. 4774. 6774. 8774.]] [10776.] 

[[ 763. 2763. 4763. 6763. 8763.]
 [ 765. 2765. 4765. 6765. 8765.]
 [ 767. 2767. 4767. 6767. 8767.]
 [ 769. 2769. 4769. 6769. 8769.]
 [ 771. 2771. 4771. 6771. 8771.]
 [ 773. 2773. 4773. 6773. 8773.]
 [ 775. 2775. 4775. 6775. 8775.]] [10777.] 

[[ 764. 2764. 4764. 6764. 87

## Real Generator using `yield`
Instead of using `return` statement, we can use `yield` statement, which is more memory efficient.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

train_x_scaler = MinMaxScaler(feature_range=(0, 1))
train_y_scaler = MinMaxScaler(feature_range=(0, 1))
val_x_scaler = MinMaxScaler(feature_range=(0, 1))
val_y_scaler = MinMaxScaler(feature_range=(0, 1))
test_x_scaler = MinMaxScaler(feature_range=(0, 1))
test_y_scaler = MinMaxScaler(feature_range=(0, 1))
 

def batch_generator(data, args, method='many_to_one',  verbose=True):
    """
    :param data: `ndarray`, input data.
    :param args: a dictionary containing values of parameters depending upon method used.
    :param method: str, default is 'many_to_one', if many_to_one, then following keys are expected in 
                   dictionary args.
            :lookback: `int`, sequence length, number of values LSTM will see at time `t` to make prediction at `t+1`.
            :in_features: `int`, number of columns in `data` starting from 0 to be considered as input
            :out_features: `int`, number of columns in `data` started from last to be considred as output/prediction.
            :trim_last_batch: bool, if True, last batch will be ignored if that contains samples less than `batch_size`.
            :norm: a dictionary which contains scaler object with which to normalize x and y data. We use separate scalers for x
                         and y data. Keys must be `x_scaler` and `y_scaler`.
            :batch_size:
            :step: step size in input data
            :min_ind: starting point from `data`
            :max_ind: end point from `data`
            :future_y_val: number of values to predict
    """
    
    many_to_one_args = {'lookback': 'required',
                        'in_features': 'required',
                        'out_features': 'required',
                        'batch_size': 'required',
                        'min_ind': 'required',
                        'max_ind': 'required',
                        'future_y_val': 'required',
                        'step': 1,
                        'norm': None,
                        'trim_last_batch':True}
    
    if method=='many_to_one':
        for k,v in many_to_one_args.items():
            if v=='required':
                if k not in args:
                    raise ValueError('for {} method, value of {} is required'.format(method, k))
                else:
                    many_to_one_args[k] = args[k]
            else:
                if k in args:
                    many_to_one_args[k] = args[k]
        
        lookback = many_to_one_args['lookback']
        in_features = many_to_one_args['in_features']
        out_features = many_to_one_args['out_features']
        batch_size = many_to_one_args['batch_size']
        min_ind = many_to_one_args['min_ind']
        max_ind = many_to_one_args['max_ind']
        future_y_val = many_to_one_args['future_y_val']
        step = many_to_one_args['step']
        norm = many_to_one_args['norm']
        trim_last_batch = many_to_one_args['trim_last_batch']

        # selecting the data of interest for x and y    
        X = data[min_ind:max_ind, 0:in_features]
        Y = data[min_ind:max_ind, -out_features:].reshape(-1,out_features)

        if norm is not None:
            x_scaler = norm['x_scaler']
            y_scaler = norm['y_scaler']
            X = x_scaler.fit_transform(X)
            Y = y_scaler.fit_transform(Y)

        # container for keeping x and y windows. A `windows` is here defined as one complete set of data at one timestep.
        x_wins = np.full((X.shape[0], lookback, in_features), np.nan, dtype=np.float32)
        y_wins = np.full((Y.shape[0], out_features), np.nan)

        # creating windows from X data
        st = lookback*step - step # starting point of sampling from data
        for j in range(st, X.shape[0]-lookback):
            en = j - lookback*step
            indices = np.arange(j, en, -step)
            ind = np.flip(indices)
            x_wins[j,:,:] = X[ind,:]

        # creating windows from Y data
        for i in range(0, Y.shape[0]-lookback):
            y_wins[i,:] = Y[i+lookback,:]



        """removing trailing nans"""
        first_nan_at_end = first_nan_from_end(y_wins[:,0])  # first nan in last part of data, start skipping from here
        y_wins = y_wins[0:first_nan_at_end,:]
        x_wins = x_wins[0:first_nan_at_end,:]

        """removing nans from start"""
        y_val = st-lookback + future_y_val
        if st>0:
            x_wins = x_wins[st:,:]
            y_wins = y_wins[y_val:,:]    

        if verbose:
            print("""shape of x data: {} \nshape of y data: {}""".format(x_wins.shape, y_wins.shape))

            print(""".\n{} values are skipped from start and {} values are skipped from end in output array"""
              .format(st, X.shape[0]-first_nan_at_end))

        pot_samples = x_wins.shape[0]

        if verbose:
            print('\npotential samples are {}'.format(pot_samples))

        residue = pot_samples % batch_size
        if verbose:
            print('\nresidue is {} '.format(residue))

        samples = pot_samples - residue
        if verbose:
            print('\nActual samples are {}'.format(samples))

        interval = np.arange(0, samples + batch_size, batch_size)
        if verbose:
            print('\nPotential intervals: {}'.format(interval ))

        interval = np.append(interval, pot_samples)
        if verbose:
            print('\nActual interval: {} '.format(interval))

        if trim_last_batch:
            no_of_batches = len(interval)-2
        else:
            no_of_batches = len(interval) - 1 

        print('\nNumber of batches are {} '.format(no_of_batches))

        # code for generator
        gen_i = 1
        while 1:

            for b in range(no_of_batches):
                st = interval[b]
                en = interval[b + 1]
                x_batch = x_wins[st:en, :, :]
                y_batch = y_wins[st:en]

                gen_i +=1

                yield x_batch, y_batch
            


_lookback=2  # sequence length
input_features = 5
output_features = 1
_batch_size = 16
input_stepsize = 2
st_ind = 0
end_ind = 600
t_plus_ith_val = 1 # which value to predict in future, e.g if input is 11,12,13,14 and default value of this variable means we
                  # want to predict 15, setting value equal to 3 means we want to predict 17.

train_args = {'lookback': _lookback,
            'in_features': input_features,
            'out_features': output_features,
            'batch_size': _batch_size,
            'min_ind': st_ind,
            'max_ind': end_ind,
            'future_y_val': t_plus_ith_val,
            'step': input_stepsize,
            'norm': {'x_scaler': train_x_scaler, 'y_scaler': train_y_scaler},
            'trim_last_batch':True}

train_gen = batch_generator(data, train_args) 

val_args = {'lookback': _lookback,
            'in_features': input_features,
            'out_features': output_features,
            'batch_size': _batch_size,
            'min_ind': 600,
            'max_ind': 800,
            'future_y_val': t_plus_ith_val,
            'step': input_stepsize,
            'norm': {'x_scaler': val_x_scaler, 'y_scaler': val_y_scaler},
            'trim_last_batch':True}

val_gen = batch_generator(data, val_args, verbose=False) 

test_args = {'lookback': _lookback,
            'in_features': input_features,
            'out_features': output_features,
            'batch_size': _batch_size,
            'min_ind': 800,
            'max_ind': 1000,
            'future_y_val': t_plus_ith_val,
            'step': input_stepsize,
            'norm': {'x_scaler': test_x_scaler, 'y_scaler': test_y_scaler},
            'trim_last_batch':True}

test_gen = batch_generator(data, test_args ) 

### Train data generator

### Validation data generator

In [ ]:
x_batch, y_batch = next(val_gen)
for inp,out in zip(x_batch, y_batch):
    print(inp,out, '\n')


Number of batches are 12 
[[0.         0.         0.         0.         0.        ]
 [0.01005025 0.01005025 0.01005025 0.01005025 0.01005025]] [0.01507538] 

[[0.00502513 0.00502513 0.00502513 0.00502513 0.00502513]
 [0.01507538 0.01507538 0.01507538 0.01507538 0.01507538]] [0.0201005] 

[[0.01005025 0.01005025 0.01005025 0.01005025 0.01005025]
 [0.0201005  0.0201005  0.0201005  0.0201005  0.0201005 ]] [0.02512563] 

[[0.01507538 0.01507538 0.01507538 0.01507538 0.01507538]
 [0.02512563 0.02512563 0.02512563 0.02512563 0.02512563]] [0.03015075] 

[[0.0201005  0.0201005  0.0201005  0.0201005  0.0201005 ]
 [0.03015075 0.03015075 0.03015075 0.03015075 0.03015075]] [0.03517588] 

[[0.02512563 0.02512563 0.02512563 0.02512563 0.02512563]
 [0.03517588 0.03517588 0.03517588 0.03517588 0.03517588]] [0.04020101] 

[[0.03015075 0.03015075 0.03015075 0.03015075 0.03015075]
 [0.040201   0.040201   0.040201   0.040201   0.040201  ]] [0.04522613] 

[[0.03517588 0.03517588 0.03517588 0.03517588 0.03

C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


### Test data generator

In [ ]:
x_batch, y_batch = next(test_gen)
for inp,out in zip(x_batch, y_batch):
    print(inp,out, '\n')

shape of x data: (196, 2, 5) 
shape of y data: (197, 1)
.
2 values are skipped from start and 2 values are skipped from end in output array

potential samples are 196

residue is 4 

Actual samples are 192

Potential intervals: [  0  16  32  48  64  80  96 112 128 144 160 176 192]

Actual interval: [  0  16  32  48  64  80  96 112 128 144 160 176 192 196] 

Number of batches are 12 
[[0.         0.         0.         0.         0.        ]
 [0.01005025 0.01005025 0.01005025 0.01005025 0.01005025]] [0.01507538] 

[[0.00502513 0.00502513 0.00502513 0.00502513 0.00502513]
 [0.01507538 0.01507538 0.01507538 0.01507538 0.01507538]] [0.0201005] 

[[0.01005025 0.01005025 0.01005025 0.01005025 0.01005025]
 [0.0201005  0.0201005  0.0201005  0.0201005  0.0201005 ]] [0.02512563] 

[[0.01507538 0.01507538 0.01507538 0.01507538 0.01507538]
 [0.02512563 0.02512563 0.02512563 0.02512563 0.02512563]] [0.03015075] 

[[0.0201005  0.0201005  0.0201005  0.0201005  0.0201005 ]
 [0.03015075 0.03015075 0.030

C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


### Test the model using keras API

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

import tensorflow

tensorflow.set_random_seed(777)  # reproducibility

In [ ]:
model = Sequential()
model.add(layers.LSTM(64, input_shape=(_lookback, input_features), dropout=0.1, recurrent_dropout=0.5,))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                            steps_per_epoch=500,
                            epochs=20,
                            validation_data=val_gen,
                            validation_steps=195)

W1220 16:37:17.659097 69052 deprecation.py:506] From C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
W1220 16:37:18.158762 69052 deprecation.py:323] From C:\Users\USER\Anaconda3\envs\tfgpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_di

Epoch 1/20
shape of x data: (596, 2, 5) 
shape of y data: (597, 1)
.
2 values are skipped from start and 2 values are skipped from end in output array

potential samples are 596

residue is 4 

Actual samples are 592

Potential intervals: [  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272
 288 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560
 576 592]

Actual interval: [  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272
 288 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560
 576 592 596] 

Number of batches are 37 
500/500 [==============================] - 6s 11ms/step - loss: 0.0540 - val_loss: 0.0615
Epoch 2/20
500/500 [==============================] - 3s 7ms/step - loss: 0.0475 - val_loss: 0.0393
Epoch 3/20
500/500 [==============================] - 3s 6ms/step - loss: 0.0344 - val_loss: 0.0532
Epoch 4/20
500/500 [==============================] - 3s 7ms/step - loss: 0.0292 - val_loss: 0.0122
Epoch 5/2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## make predictions

In [ ]:
test_x, test_y = next(test_gen)
test_prediction = model.predict(test_x, 1, verbose=True)
test_predict = test_y_scaler.inverse_transform(test_prediction)
true = test_y_scaler.inverse_transform(test_y)
for t,p in zip(true, test_predict):
    print(t,p)